# Data Cleaning

This notebook focuses on cleaning the tables from the Northwind database.  
The objective is to:

- Remove duplicates and irrelevant columns  
- Standardize data types  
- Fix formatting issues and inconsistencies  
- Handle missing values appropriately  

In [2]:
import sqlite3
import pandas as pd
import os

#create new folder "cleaned" for the cleaned tables
os.makedirs('cleaned', exist_ok=True)

#connect to SQLite database
conn = sqlite3.connect("../data/northwind.db")

query = "SELECT name FROM sqlite_master WHERE type='table';"
tables_df = pd.read_sql_query(query, conn)
tables_df

,name
0,Categories
1,sqlite_sequence
2,CustomerCustomerDemo
3,CustomerDemographics
4,Customers
5,Employees
6,EmployeeTerritories
7,Order Details
8,Orders
9,Products


## Categories

From the exploration phase, we concluded that the following actions are to be made on this table:
> - The column (`Picture`) is irrelevant and will be dropped in the cleaning phase.  
> - `object` columns will be converted to `string` for consistency.

In [42]:
categories = pd.read_sql_query("SELECT * FROM Categories;", conn)

categories_cl = categories.drop(columns=['Picture'])

#change types
categories_cl['CategoryName'] = categories_cl['CategoryName'].astype('string')
categories_cl['Description'] = categories_cl['Description'].astype('string')

In [43]:
categories_cl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   CategoryID    8 non-null      int64 
 1   CategoryName  8 non-null      string
 2   Description   8 non-null      string
dtypes: int64(1), string(2)
memory usage: 324.0 bytes


In [44]:
# save the cleaned file in the new folder
categories_cl.to_csv('cleaned/categories.csv', index=False)

## Products

From the exploration phase, we concluded that the following actions are to be made on this table:
> -  Convert `Discontinued` to integer, `ProductName` to string for consistency 

In [5]:
products = pd.read_sql_query("SELECT * FROM Products;", conn)
products_cl = products.copy()
#convert columns to appropriate types
products_cl['Discontinued'] = products_cl['Discontinued'].astype(int)
products_cl['ProductName'] = products_cl['ProductName'].astype('string')
products_cl['QuantityPerUnit'] = products_cl['QuantityPerUnit'].astype('string')

In [6]:
products_cl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ProductID        77 non-null     int64  
 1   ProductName      77 non-null     string 
 2   SupplierID       77 non-null     int64  
 3   CategoryID       77 non-null     int64  
 4   QuantityPerUnit  77 non-null     string 
 5   UnitPrice        77 non-null     float64
 6   UnitsInStock     77 non-null     int64  
 7   UnitsOnOrder     77 non-null     int64  
 8   ReorderLevel     77 non-null     int64  
 9   Discontinued     77 non-null     int64  
dtypes: float64(1), int64(7), string(2)
memory usage: 6.1 KB


In [7]:
#save the file
products_cl.to_csv('cleaned/products.csv', index=False)

## Order Details

> No data cleaning needed.

In [3]:
order_details = pd.read_sql_query('SELECT * FROM "Order Details";', conn)

order_details.to_csv('cleaned/order_details.csv', index=False)

## Orders

From the exploration phase, we concluded that the following actions are to be made on this table:
> - Drop the columns `ShipAddress` and `ShipPostalCode` due to irrelevance and missing data
> - Convert `OrderDate`, `RequiredDate`, `ShippedDate` to datetime
> - Convert `CustomerID`, `ShipName`, `ShipCity`, `ShipRegion`, `ShipCountry` to string

In [48]:
orders = pd.read_sql_query("SELECT * FROM Orders;", conn)

#drop irrelevant columns
orders_cl = orders.drop(columns=['ShipAddress', 'ShipPostalCode'])

# Convert date columns to datetime
date_cols = ['OrderDate', 'RequiredDate', 'ShippedDate']
orders_cl[date_cols] = orders_cl[date_cols].apply(pd.to_datetime, format='mixed')

# Convert selected object columns to string
string_cols = ['CustomerID', 'ShipName', 'ShipCity', 'ShipRegion', 'ShipCountry']
orders_cl[string_cols] = orders_cl[string_cols].astype('string')

In [49]:
orders_cl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16282 entries, 0 to 16281
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   OrderID       16282 non-null  int64         
 1   CustomerID    16282 non-null  string        
 2   EmployeeID    16282 non-null  int64         
 3   OrderDate     16282 non-null  datetime64[ns]
 4   RequiredDate  16282 non-null  datetime64[ns]
 5   ShippedDate   16261 non-null  datetime64[ns]
 6   ShipVia       16282 non-null  int64         
 7   Freight       16282 non-null  float64       
 8   ShipName      16282 non-null  string        
 9   ShipCity      16282 non-null  string        
 10  ShipRegion    16282 non-null  string        
 11  ShipCountry   16282 non-null  string        
dtypes: datetime64[ns](3), float64(1), int64(3), string(5)
memory usage: 1.5 MB


In [50]:
#save
orders_cl.to_csv('cleaned/orders.csv', index=False)

## Customers

From the exploration phase, we concluded that the following actions are to be made on this table:
> - Drop the `Address`, `PostalCode`, `Phone` and `Fax` columns (many missing values, not useful)
> - Convert all columns to `string` for consistency and Power BI compatibility
> - Replace missing values in `City`, `Region`, and `Country` with `'Unknown'` to avoid nulls in Power BI

In [51]:
# Load the table
customers = pd.read_sql_query("SELECT * FROM Customers;", conn)

# Drop irrelevant columns
customers_cl = customers.drop(columns=['Address', 'PostalCode', 'Phone', 'Fax'])

# Convert all columns to string
customers_cl = customers_cl.astype('string')

# Replace missing values in City, Region, and Country
customers_cl[['City', 'Region', 'Country']] = customers_cl[['City', 'Region', 'Country']].fillna('Unknown')

In [52]:
customers_cl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93 entries, 0 to 92
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   CustomerID    93 non-null     string
 1   CompanyName   93 non-null     string
 2   ContactName   93 non-null     string
 3   ContactTitle  93 non-null     string
 4   City          93 non-null     string
 5   Region        93 non-null     string
 6   Country       93 non-null     string
dtypes: string(7)
memory usage: 5.2 KB


In [53]:
# Save the cleaned file
customers_cl.to_csv('cleaned/customers.csv', index=False)

## Suppliers

From the exploration phase, we concluded that the following actions are to be made on this table:
> - Drop the `Address`, `PostalCode`, `Phone`, `Fax` and `HomePage` columns (not useful)
> - Convert all columns to `string` for consistency and Power BI compatibility  
> - Replace missing value in `Region` with `'Unknown'`

In [54]:
suppliers = pd.read_sql_query("SELECT * FROM Suppliers;", conn)

suppliers_cl = suppliers.drop(columns=['Address', 'PostalCode', 'Phone', 'Fax', 'HomePage'])

# Replace missing values in Region
suppliers_cl['Region'] = suppliers_cl['Region'].fillna('Unknown')

suppliers_cl = suppliers_cl.astype('string')

In [55]:
suppliers_cl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   SupplierID    29 non-null     string
 1   CompanyName   29 non-null     string
 2   ContactName   29 non-null     string
 3   ContactTitle  29 non-null     string
 4   City          29 non-null     string
 5   Region        29 non-null     string
 6   Country       29 non-null     string
dtypes: string(7)
memory usage: 1.7 KB


In [56]:
suppliers_cl.to_csv('cleaned/suppliers.csv', index=False)

## Employees

From the exploration phase, we concluded that the following actions are to be made on this table:
> - Drop the `TitleOfCourtesy`, `BirthDate`, `Address`, `PostalCode`, `HomePhone`, `Extension`, `Photo`, `Notes`, and `PhotoPath` columns (not useful)  
> - Convert `HireDate` to datetime  
> - Convert all `object` columns to `string` for consistency and Power BI compatibility  
> - Merge the `FirstName` and `LastName` columns  
> - Replace missing value in `ReportsTo` with `-1` (likely top-level manager)


In [57]:
employees = pd.read_sql_query("SELECT * FROM Employees;", conn)

employees_cl = employees.drop(columns=[
    'TitleOfCourtesy', 'BirthDate', 'Address', 'PostalCode',
    'HomePhone', 'Extension', 'Photo', 'Notes', 'PhotoPath'
])

# Convert HireDate to datetime
employees_cl['HireDate'] = pd.to_datetime(employees_cl['HireDate'], format='mixed')

# Merge FirstName and LastName, place after EmployeeID
employees_cl.insert(loc=1, column='FullName', value=employees_cl['FirstName'] + ' ' + employees_cl['LastName'])
employees_cl = employees_cl.drop(columns=['FirstName', 'LastName'])

# Replace missing value in ReportsTo with -1
employees_cl['ReportsTo'] = employees_cl['ReportsTo'].fillna(-1).astype(int)

# Convert all object columns to string
for col in employees_cl.select_dtypes(include='object').columns:
    employees_cl[col] = employees_cl[col].astype('string')

In [58]:
employees_cl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   EmployeeID  9 non-null      int64         
 1   FullName    9 non-null      string        
 2   Title       9 non-null      string        
 3   HireDate    9 non-null      datetime64[ns]
 4   City        9 non-null      string        
 5   Region      9 non-null      string        
 6   Country     9 non-null      string        
 7   ReportsTo   9 non-null      int64         
dtypes: datetime64[ns](1), int64(2), string(5)
memory usage: 708.0 bytes


In [59]:
# Save cleaned file
employees_cl.to_csv('cleaned/employees.csv', index=False)

## Shippers

From the exploration phase, we concluded that the following actions are to be made on this table:
> - `Phone` column to be removed


In [60]:
shippers = pd.read_sql_query("SELECT * FROM Shippers;", conn)

shippers_cl = shippers.drop(columns=['Phone'])

shippers_cl = shippers_cl.astype('string')

In [61]:
shippers_cl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ShipperID    3 non-null      string
 1   CompanyName  3 non-null      string
dtypes: string(2)
memory usage: 180.0 bytes


In [62]:
shippers_cl.to_csv('cleaned/shippers.csv', index=False)